In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Folder_path = "/content/drive/MyDrive/Dataset"


In [ ]:
# Cell 1: Install required packages
!pip install tensorflow scikit-image opencv-python matplotlib

In [ ]:
%%writefile autoencoder_training.py
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.metrics import Mean
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
import cv2
from google.colab import files

# Custom hybrid loss function (SSIM + MSE)
def hybrid_loss(y_true, y_pred):
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))
    return mse + ssim_loss

class AutoencoderTrainer:
    def __init__(self, img_size=(256, 256), latent_dim=128):
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.model = self.build_autoencoder()
        self.errors = []
        self.threshold = None  # Initialize threshold
        # Print model summary
        print("\nModel Summary:")
        self.model.summary()  # Added to display model architecture and parameters

    def build_autoencoder(self):
        # Encoder
        inputs = layers.Input(shape=(self.img_size[0], self.img_size[1], 3))
        x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
        x = layers.MaxPooling2D(2)(x)
        x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
        x = layers.MaxPooling2D(2)(x)
        x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
        x = layers.MaxPooling2D(2)(x)

        # Latent space
        x = layers.Flatten()(x)
        latent = layers.Dense(self.latent_dim, name='latent')(x)

        # Decoder
        x = layers.Dense(32 * 32 * 128)(latent)
        x = layers.Reshape((32, 32, 128))(x)
        x = layers.Conv2DTranspose(128, 3, activation='relu', padding='same')(x)
        x = layers.UpSampling2D(2)(x)
        x = layers.Conv2DTranspose(64, 3, activation='relu', padding='same')(x)
        x = layers.UpSampling2D(2)(x)
        x = layers.Conv2DTranspose(32, 3, activation='relu', padding='same')(x)
        x = layers.UpSampling2D(2)(x)
        outputs = layers.Conv2D(3, 3, activation='sigmoid', padding='same')(x)

        model = Model(inputs, outputs)
        model.compile(optimizer='adam', loss=hybrid_loss)
        return model

    def load_data(self, data_dir, mask_dir):
        images = []
        for disease_folder in os.listdir(data_dir):
            img_path = os.path.join(data_dir, disease_folder)
            mask_path = os.path.join(mask_dir, disease_folder)

            for img_file in os.listdir(img_path):
                if img_file.endswith('.jpg'):
                    # Load and preprocess image
                    img = load_img(os.path.join(img_path, img_file), target_size=self.img_size)
                    img = img_to_array(img) / 255.0

                    # Load corresponding mask
                    mask_file = img_file.replace('.jpg', '_mask.png')
                    mask = load_img(os.path.join(mask_path, mask_file),
                                    target_size=self.img_size,
                                    color_mode='rgb')
                    mask = img_to_array(mask) / 255.0

                    # Combine image and mask
                    combined = np.concatenate([img, mask], axis=2)[:, :, :3]
                    images.append(combined)

        return np.array(images)

    def train(self, data_dir, mask_dir, epochs=50, batch_size=32):
        # Load data
        X = self.load_data(data_dir, mask_dir)

        # Training history
        history = {'loss': []}

        # Train the model
        for epoch in range(epochs):
            print(f"\nEpoch {epoch+1}/{epochs}")
            hist = self.model.fit(X, X,
                                  batch_size=batch_size,
                                  epochs=1,
                                  verbose=1)

            # Calculate reconstruction error
            reconstructed = self.model.predict(X)
            error = np.mean([hybrid_loss(X[i:i+1], reconstructed[i:i+1]).numpy()
                            for i in range(len(X))])
            self.errors.append(error)

            # Store history
            history['loss'].append(hist.history['loss'][0])
            print(f"Reconstruction Error: {error:.4f}")

        # Calculate threshold
        self.threshold = np.mean(self.errors)
        print(f"Threshold value: {self.threshold:.4f}")

        # Plot training history
        plt.figure(figsize=(10, 5))
        plt.plot(history['loss'])
        plt.title('Autoencoder Training Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.grid(True)
        plt.savefig('autoencoder_loss.png')
        files.download('autoencoder_loss.png')
        plt.close()

        # Save model
        self.model.save('autoencoder_model.h5')
        files.download('autoencoder_model.h5')

        # Save threshold to a file
        np.save('threshold.npy', self.threshold)

        return history

    def preprocess_image(self, img_path, mask_path=None):
        # Load and preprocess image
        img = load_img(img_path, target_size=self.img_size)
        img = img_to_array(img) / 255.0

        if mask_path:
            mask = load_img(mask_path, target_size=self.img_size, color_mode='rgb')
            mask = img_to_array(mask) / 255.0
            combined = np.concatenate([img, mask], axis=2)[:, :, :3]
        else:
            combined = img

        return np.expand_dims(combined, axis=0)

# Run this when the script is executed directly
if __name__ == "__main__":
    trainer = AutoencoderTrainer()


Model Summary:


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent (Dense)                  │ (None, 128)            │    16,777,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 131072)         │    16,908,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 32, 32, 128)    │       147,584 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_4              │ (None, 64, 64, 64)     │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_4 (UpSampling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_5              │ (None, 128, 128, 32)   │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_5 (UpSampling2D)  │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 256, 256, 3)    │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,019,587 (129.77 MB)

 Trainable params: 34,019,587 (129.77 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
%%writefile encoder_features.py
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

class FeatureExtractor:
    def __init__(self, autoencoder_path='/content/drive/MyDrive/Dataset/autoencoder_model.h5'):
        # Load the full autoencoder
        autoencoder = tf.keras.models.load_model(autoencoder_path,
                                               custom_objects={'hybrid_loss': lambda x, y: x})

        # Create encoder model from the latent layer
        self.encoder = Model(inputs=autoencoder.input,
                           outputs=autoencoder.get_layer('latent').output)

        # Print encoder model summary
        print("\nEncoder Model Summary:")
        self.encoder.summary()

        self.img_size = (256, 256)

    def preprocess_image(self, img_path, mask_path=None):
        # Load and preprocess image
        img = load_img(img_path, target_size=self.img_size)
        img = img_to_array(img) / 255.0

        if mask_path:
            mask = load_img(mask_path, target_size=self.img_size, color_mode='rgb')
            mask = img_to_array(mask) / 255.0
            combined = np.concatenate([img, mask], axis=2)[:, :, :3]
        else:
            combined = img

        return np.expand_dims(combined, axis=0)

    def extract_features(self, img_path, mask_path=None):
        preprocessed = self.preprocess_image(img_path, mask_path)
        features = self.encoder.predict(preprocessed)
        return features.flatten()

# Run this when the script is executed directly
if __name__ == "__main__":
    extractor = FeatureExtractor()


Encoder Model Summary:


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent (Dense)                  │ (None, 128)            │    16,777,344 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,870,592 (64.36 MB)

 Trainable params: 16,870,592 (64.36 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
%%writefile classifier_training.py
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
from encoder_features import FeatureExtractor
import matplotlib.pyplot as plt
from google.colab import files

class DiseaseClassifier:
    def __init__(self, feature_dim=128, num_classes=6, model_path=None):
        self.feature_extractor = FeatureExtractor()
        if model_path:
            self.model = tf.keras.models.load_model(model_path)
            self.class_names = np.load('/content/drive/MyDrive/Dataset/class_names.npy', allow_pickle=True).tolist()
        else:
            self.model = self.build_classifier(feature_dim, num_classes)
            self.class_names = []

        # Print classifier model summary
        print("\nClassifier Model Summary:")
        self.model.summary()

    def build_classifier(self, feature_dim, num_classes):
        inputs = layers.Input(shape=(feature_dim,))
        x = layers.Dense(256, activation='relu')(inputs)
        x = layers.Dropout(0.3)(x)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.Dropout(0.3)(x)
        outputs = layers.Dense(num_classes, activation='softmax')(x)

        model = Model(inputs, outputs)
        model.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
        return model

    def load_data(self, data_dir, mask_dir):
        X, y = [], []
        self.class_names = sorted(os.listdir(data_dir))

        for idx, disease in enumerate(self.class_names):
            img_path = os.path.join(data_dir, disease)
            mask_path = os.path.join(mask_dir, disease)

            for img_file in os.listdir(img_path):
                if img_file.endswith('.jpg'):
                    mask_file = img_file.replace('.jpg', '_mask.png')
                    features = self.feature_extractor.extract_features(
                        os.path.join(img_path, img_file),
                        os.path.join(mask_path, mask_file)
                    )
                    X.append(features)
                    y.append(idx)

        return np.array(X), tf.keras.utils.to_categorical(y, len(self.class_names))

    def train(self, data_dir, mask_dir, epochs=50, batch_size=32):
        X, y = self.load_data(data_dir, mask_dir)
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

        history = self.model.fit(X_train, y_train,
                               validation_data=(X_val, y_val),
                               epochs=epochs,
                               batch_size=batch_size,
                               verbose=1)

        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)

        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.savefig('classifier_metrics.png')
        files.download('classifier_metrics.png')
        plt.close()

        self.model.save('classifier_model.h5')
        files.download('classifier_model.h5')

        np.save('class_names.npy', self.class_names)
        files.download('class_names.npy')

        return history

    def predict(self, img_path, mask_path):
        if not self.class_names:
            raise ValueError("Class names not initialized. Train the model or load a trained model first.")
        features = self.feature_extractor.extract_features(img_path, mask_path)
        prediction = self.model.predict(np.expand_dims(features, axis=0))
        return self.class_names[np.argmax(prediction)]

# Run this when the script is executed directly
if __name__ == "__main__":
    classifier = DiseaseClassifier()


Encoder Model Summary:


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent (Dense)                  │ (None, 128)            │    16,777,344 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,870,592 (64.36 MB)

 Trainable params: 16,870,592 (64.36 MB)

...

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 66,694 (260.52 KB)

 Trainable params: 66,694 (260.52 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from autoencoder_training import AutoencoderTrainer

# Train autoencoder
trainer = AutoencoderTrainer()
trainer.train('/content/drive/MyDrive/Dataset/dataset', '/content/drive/MyDrive/Dataset/dataset_mask', epochs=5, batch_size=32)


Epoch 1/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 227s 16s/step - loss: 0.6195
14/14 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step
Reconstruction Error: 0.4178

Epoch 2/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 222s 16s/step - loss: 0.4051
14/14 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step
Reconstruction Error: 0.3079

Epoch 3/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 221s 16s/step - loss: 0.2979
14/14 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step
Reconstruction Error: 0.2872

Epoch 4/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 221s 16s/step - loss: 0.2772
14/14 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step
Reconstruction Error: 0.2625

Epoch 5/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 221s 16s/step - loss: 0.2516
14/14 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step
Reconstruction Error: 0.2523
Threshold value: 0.3055


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

{'loss': [0.559209406375885,
  0.38523921370506287,
  0.29021480679512024,
  0.2711125314235687,
  0.2565198838710785]}

In [ ]:
from classifier_training import DiseaseClassifier

# Train classifier (after autoencoder training is complete)
classifier = DiseaseClassifier()
classifier.train('/content/drive/MyDrive/Dataset/dataset', '/content/drive/MyDrive/Dataset/dataset_mask', epochs=300, batch_size=32)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━

In [ ]:
import numpy as np
from autoencoder_training import AutoencoderTrainer, hybrid_loss
from classifier_training import DiseaseClassifier
from tensorflow.keras.models import load_model

def test_pipeline(img_path, mask_path):
    # Load trained autoencoder model
    autoencoder = load_model('/content/drive/MyDrive/Dataset/autoencoder_model.h5', custom_objects={'hybrid_loss': hybrid_loss})

    # Load threshold
    threshold = np.load('/content/drive/MyDrive/Dataset/threshold.npy')

    # Load trained classifier with saved model and class names
    classifier = DiseaseClassifier(model_path='/content/drive/MyDrive/Dataset/classifier_model (8).h5')

    # Preprocess test image
    trainer = AutoencoderTrainer()  # For preprocessing only
    img = trainer.preprocess_image(img_path, mask_path)

    # Check if known or unknown disease
    reconstructed = autoencoder.predict(img)
    error = hybrid_loss(img, reconstructed).numpy()

    print(f"Reconstruction Error: {error:.4f}")
    print(f"Threshold: {threshold:.4f}")

    if error > threshold:
        print("Unknown disease detected")
        return "Unknown"
    else:
        print("Known disease detected")
        prediction = classifier.predict(img_path, mask_path)
        print(f"Predicted disease: {prediction}")
        return prediction

# Test with an example
result = test_pipeline('/content/img_0001.jpg',
                      '/content/img_0001_mask.png')


Encoder Model Summary:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent (Dense)                  │ (None, 128)            │    16,777,344 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,870,592 (64.36 MB)

 Trainable params: 16,870,592 (64.36 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent (Dense)                  │ (None, 128)            │    16,777,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 131072)         │    16,908,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 32, 32, 128)    │       147,584 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 64, 64, 64)     │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 128, 128, 32)   │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 256, 256, 3)    │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,019,587 (129.77 MB)

 Trainable params: 34,019,587 (129.77 MB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 547ms/step
Reconstruction Error: 0.1915
Threshold: 0.2240
Known disease detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Predicted disease: 00. Normal
